# Flight Parquet Multi-Year Exploration

Following `explore_year_parquet_plan.md`, this notebook loads all available Parquet files from `~/flight_data` and performs exploratory analysis across years.


In [1]:
from pathlib import Path
from typing import List

import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")


## Load Parquet Files


In [ ]:
data_dir = Path.home() / "flight_data"
parquet_paths = sorted(data_dir.glob("*.parquet"))

if not parquet_paths:
    raise FileNotFoundError(f"No Parquet files found in {data_dir}")

print("Discovered Parquet files:")
for path in parquet_paths:
    print(f"- {path.name}")

# Read with dask - can read multiple files efficiently
# Add YEAR from filename
frames: List[dd.DataFrame] = []
for path in parquet_paths:
    df = dd.read_parquet(path)
    df = df.assign(YEAR=path.stem)
    frames.append(df)

flights_all = dd.concat(frames, ignore_index=True)

# IMPORTANT: Compute n_rows ONCE and reuse it
# Store it in a variable that other cells can access
n_rows = len(flights_all)  # This scans all partitions - expensive!
n_cols = flights_all.shape[1]
print(f"\nCombined DataFrame shape: ({n_rows:,}, {n_cols})")
print(f"Number of partitions: {flights_all.npartitions}")
flights_all.head()


Discovered Parquet files:
- 2009.parquet
- 2010.parquet
- 2011.parquet
- 2012.parquet
- 2013.parquet
- 2014.parquet
- 2015.parquet
- 2016.parquet
- 2017.parquet
- 2018.parquet

Combined DataFrame shape: (61,556,964, 29)


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27,YEAR
0,2009-01-01,XE,1204,DCA,EWR,1100,1058.0,-2.0,18.0,1116.0,...,68.0,42.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN,2009
1,2009-01-01,XE,1206,EWR,IAD,1510,1509.0,-1.0,28.0,1537.0,...,75.0,43.0,213.0,NaN,NaN,NaN,NaN,NaN,NaN,2009
2,2009-01-01,XE,1207,EWR,DCA,1100,1059.0,-1.0,20.0,1119.0,...,62.0,36.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN,2009
3,2009-01-01,XE,1208,DCA,EWR,1240,1249.0,9.0,10.0,1259.0,...,56.0,37.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN,2009
4,2009-01-01,XE,1209,IAD,EWR,1715,1705.0,-10.0,24.0,1729.0,...,77.0,40.0,213.0,NaN,NaN,NaN,NaN,NaN,NaN,2009


**Note on Performance**: For data that fits in memory (like this ~14GB dataset), pandas is typically **faster than dask** because:

1. **No overhead**: Pandas uses optimized C code without task scheduling overhead
2. **Single-threaded optimization**: Pandas operations are highly optimized for single-threaded execution
3. **Memory locality**: All data in one place enables better cache utilization

**Dask is beneficial when**:
- Data doesn't fit in memory (larger than RAM)
- You have many CPU cores to parallelize across
- You need distributed computing across machines
- You can batch multiple operations together

The notebook is optimized to minimize redundant `len()` calls which scan the entire dataset.


## Basic Statistics


In [ ]:
# Reuse n_rows from cell 3 to avoid recomputing
# If running this cell independently, uncomment the line below:
# n_rows = len(flights_all)

row_count = n_rows  # Reuse from previous cell
column_count = flights_all.shape[1]
print(f"Rows: {row_count:,}")
print(f"Columns: {column_count}")

print("\nColumns:")
print(", ".join(flights_all.columns))

print("\nDtypes:")
print(flights_all.dtypes.sort_index())


Rows: 61,556,964
Columns: 29

Columns:
FL_DATE, OP_CARRIER, OP_CARRIER_FL_NUM, ORIGIN, DEST, CRS_DEP_TIME, DEP_TIME, DEP_DELAY, TAXI_OUT, WHEELS_OFF, WHEELS_ON, TAXI_IN, CRS_ARR_TIME, ARR_TIME, ARR_DELAY, CANCELLED, CANCELLATION_CODE, DIVERTED, CRS_ELAPSED_TIME, ACTUAL_ELAPSED_TIME, AIR_TIME, DISTANCE, CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY, Unnamed: 27, YEAR

Dtypes:
ACTUAL_ELAPSED_TIME    float64
AIR_TIME               float64
ARR_DELAY              float64
ARR_TIME               float64
CANCELLATION_CODE       object
CANCELLED              float64
CARRIER_DELAY          float64
CRS_ARR_TIME             Int64
CRS_DEP_TIME             Int64
CRS_ELAPSED_TIME       float64
DEP_DELAY              float64
DEP_TIME               float64
DEST                    object
DISTANCE               float64
DIVERTED               float64
FL_DATE                 object
LATE_AIRCRAFT_DELAY    float64
NAS_DELAY              float64
OP_CARRIER              object
OP

In [ ]:
numeric_summary = flights_all.describe().T.compute()
numeric_summary


In [ ]:
categorical_columns = [
    col
    for col in ["YEAR", "OP_CARRIER", "ORIGIN", "DEST", "TAIL_NUM"]
    if col in flights_all.columns
]

# Compute value_counts for all columns at once (batched)
categorical_summaries = {}
for column in categorical_columns:
    # .head(10) is lazy - need .compute() to get actual results
    categorical_summaries[column] = flights_all[column].value_counts(dropna=False).head(10).compute()

categorical_summaries


## Value Distributions


In [ ]:
numeric_columns_for_hist = [col for col in ["DEP_DELAY", "ARR_DELAY", "DISTANCE"] if col in flights_all.columns]

# Reuse n_rows from cell 3 to avoid recomputing
# If running this cell independently, uncomment:
# n_rows = len(flights_all)

sample_size = min(500_000, n_rows)
flights_sample = flights_all.sample(frac=sample_size / n_rows, random_state=42).compute()

fig, axes = plt.subplots(len(numeric_columns_for_hist), 1, figsize=(8, 4 * len(numeric_columns_for_hist)), constrained_layout=True)
if len(numeric_columns_for_hist) == 1:
    axes = [axes]

for ax, column in zip(axes, numeric_columns_for_hist):
    sns.histplot(data=flights_sample, x=column, ax=ax, bins=80, kde=False, edgecolor="black")
    ax.set_title(f"Distribution of {column} (sample of {len(flights_sample):,} rows)")
    ax.set_xlabel(column)
    ax.set_ylabel("Count")
    ax.set_xlim(left=min(0, flights_sample[column].min(skipna=True)))

plt.show()


## Missing Value Analysis


In [ ]:
missing_counts = flights_all.isna().sum().compute()

# Reuse n_rows from cell 3 - DON'T recompute!
# If running this cell independently, uncomment:
# n_rows = len(flights_all)

missing_percent = (missing_counts / n_rows) * 100

import pandas as pd
missing_df = pd.DataFrame({
    "missing_count": missing_counts,
    "missing_percent": missing_percent,
})
missing_df = missing_df.sort_values(by="missing_percent", ascending=False)
missing_df


In [ ]:
top_missing = missing_df[missing_df["missing_count"] > 0].head(20)

plt.figure(figsize=(10, 6))
sns.barplot(
    data=top_missing.reset_index(),
    x="missing_percent",
    y="index",
    palette="Blues_r"
)
plt.title("Top Columns by Missing Percentage")
plt.xlabel("Missing %")
plt.ylabel("Column")
plt.show()


## Notes & Next Steps
- Review categorical summaries for notable route or carrier patterns.
- Dive deeper into delay distributions by airline or airport using groupby operations.
- Extend the analysis with year-over-year comparisons or integrate external datasets (e.g., weather) for richer insights.
